In [1]:
import lib.aoc.grid2d.grid as grid2d
import lib.aoc.grid2d.vector as vector2d
from collections import deque, defaultdict
from itertools import count

In [2]:
with open("../data/2024/day12.txt") as f:
    data = f.read()

In [3]:
grid = grid2d.parse(data)

In [4]:
def harvest(crop: str, start_at: tuple) -> set:
    queue = deque([start_at])
    visited = set()

    # Group adjacent crop tiles with a BFS flood fill
    while queue:
        at = queue.popleft()
        if at in visited: continue
        visited.add(at)

        # Visit matching neighbor crop tiles
        for n_at, n_crop in grid2d.neighbors(grid, at, directions=vector2d.NESW):
            if n_crop != crop or n_at in visited: continue
            queue.append(n_at)

    return visited

In [5]:
plots = []
harvested = set()

# For every crop tile
for yx, crop in grid.items():
    if yx in harvested: continue
    # If not yet harvested
    new_harvest = harvest(crop, yx)
    # Store distinct regions as a set of coords
    plots.append((crop, new_harvest))
    # Union this harvest with all previous ones
    harvested = harvested | new_harvest

In [6]:
part1, part2 = 0, 0
edge_bits = [1,2,4,8] # N,E,S,W edges

# For every distinct crop region
for crop, coords in plots:
    contiguous_sides, open_sides = 0, 0
    edges = defaultdict(int)

    # Sort the coordinates, so we move left-to-right, top-to-bottom
    for coord in sorted(coords):
        neighbors = [(grid.get(vector2d.add(coord, v))) for v in vector2d.NESW]

        # neighbor, vector, interval
        for n, v, i in zip(neighbors, vector2d.NESW, count()):
            if n == crop:
                open_sides += 1

            # Merge contiguous open sides
            if n != crop:
                edges[coord] += edge_bits[i]

                # If it's a north/south edge and the west neighbor is contiguous
                if i in (0,2) and edges.get(vector2d.add(coord, vector2d.NESW[3]), 0) & edge_bits[i]:
                    pass
                # If it's an east/west edge and the north neighbor is contiguous
                elif i in (1,3) and edges.get(vector2d.add(coord, vector2d.NESW[0]), 0) & edge_bits[i]:
                    pass
                else:
                    contiguous_sides += 1

    # Our simple perimeter assumes 4 sides per coordinate, minus open sides
    part1 += len(coords) * (4*len(coords)-open_sides)
    part2 += len(coords) * contiguous_sides

print(f"Part 1: {part1}")
print(f"Part 2: {part2}")

Part 1: 1344578
Part 2: 814302
